<a href="https://colab.research.google.com/github/ygGao1120/Reinforcement_Learning/blob/main/Reinforcement_Learning_ES_compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

%%bash

# install required system dependencies
apt-get install -y xvfb x11-utils

# install required python dependencies (might need to install additional gym extras depending)
pip install gym[box2d]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Reading package lists...
Building dependency tree...
Reading state information...
x11-utils is already the newest version (7.7+3build1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.9).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
  Created wheel for PyOpenGL-accelerate: filename=PyOpenGL_accelerate-3.1.5-cp37-cp37m-linux_x86_64.whl size=1599552 sha256=5bf3a1140a90e320b8a8583677a5169722267ffd41a555ebd7d8449abe573c45
  Stored in directory: /root/.cache/pip/wheels/1c/f5/6f/169afb3f2d476c5e807f8515b3c9bc9b819c3962316aa804eb
Successfully built PyOpenGL-accelerate


In [ ]:
import pyvirtualdisplay


_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_ = _display.start()

In [ ]:
import gym
import numpy as np
#import nevergrad as ng
import cv2
from scipy.optimize import minimize
import scipy
import scipy.stats as st
from scipy.linalg import *

import matplotlib.pyplot as plt
from matplotlib import animation

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

import pandas as pd

In [ ]:
max_num_step = 1000

In [ ]:
class Solution:
    def __init__(self, env: gym.Env, grad_method='FD', N=8, sigma=0.1,T = 200):
        self.obs_dim = env.observation_space.shape
        self.act_dim = env.action_space.n
        print(self.obs_dim, self.act_dim)
        self.params_shape = self.act_dim * self.obs_dim[0]
        self.params_dot_shape = self.act_dim, self.obs_dim[0]
        self.N = N
        self.sigma = sigma
        self.grad_method_name = grad_method
        self.T = T
        self.R = np.random.RandomState(seed = 40)
        self.initial = self.R.rand(self.params_shape)

    def fun(self, params: np.ndarray) -> float:
        env = gym.make('CartPole-v1')
        total_reward = 0.0
        total_steps = 0
        obs = env.reset()
        params = params.reshape(self.params_dot_shape)
        for t in range(max_num_step):

            a = np.dot(params, obs)
            a = np.argmax(a.flatten())
            
            obs, reward, done, _ = env.step(a)  # take a random action
            total_reward += reward
            total_steps += 1
            if done:
                break
        # print("Episode done in %d steps, total reward %.2f" % (total_steps, total_reward))
        # print(total_reward)
        return -total_reward

    def gradient_term(self, params):
        if self.grad_method_name == "FD":
            return self.FD_grad(params)
        elif self.grad_method_name == "antithetic":
            return self.antithetic_grad(params)
        elif self.grad_method_name == "vanilla":
            return self.vanilla_grad(params)
        elif self.grad_method_name == "lasso":
            return self.lasso(self,params)
        elif self.grad_method_name == "ridge":
            return self.ridge(self,params)
        elif self.grad_method_name == "LP":
            return self.lp_gra(params)
        else:
            raise ValueError("Unknown gradient method")

    def grad_estimate_method(self):
        pass

    @staticmethod
    def ridge(self,params):
        F = self.fun(params)
        y = np.zeros(self.N)
        #generate noise with different methods
        noise = self.independent_gau(self.N)
        for i in range(self.N):
            y[i] = self.fun(params+noise[i,:])-F

        SSmodel=StandardScaler()
        SSmodel.fit(noise)
        Train_x=SSmodel.transform(noise)
        model = Ridge(fit_intercept = False)
        model.fit(Train_x,y)
        return model.coef_/np.std(noise,axis = 0)


    @staticmethod
    def lasso(self,params):
        F = self.fun(params)
        y = np.zeros(self.N)
        #generate noise with different methods
        noise = self.independent_gau(self.N)
        for i in range(self.N):
            y[i] = self.fun(params+noise[i,:])-F

        SSmodel=StandardScaler()
        SSmodel.fit(noise)
        Train_x=SSmodel.transform(noise)
        model = Lasso(fit_intercept = False)
        model.fit(Train_x,y)
        return model.coef_/np.std(noise,axis = 0)
    
    def lp_gra(self,params):
        F = self.fun(params)
        y = np.zeros(self.N)
        #generate noise with different methods
        noise = self.independent_gau(self.N)
        for i in range(self.N):
            y[i] = self.fun(params+noise[i,:])-F

        def lp_decoding(z):
            res=np.abs(y - np.dot(noise[i,:],z))
            return np.sum(res)

        curr_opt = scipy.optimize.minimize(lp_decoding,np.zeros(self.params_shape),tol = 0.01)
        return curr_opt.x

    # monte carlo based gradient
    def vanilla_grad(self, params: np.ndarray) -> np.ndarray:
        grad = np.zeros(params.shape)
        # generate noise with different methods
        #noise = self.independent_gau(self.N)
        # noise = self.orth_gau()
        noise = self.hadamard()
        #noise = self.random_givens()

        for i in range(self.N):
            curr_noise = noise[i]
            theta = params + self.sigma * curr_noise
            F = self.fun(theta)
            grad += F * curr_noise
        grad = grad / (self.N * self.sigma)
        return grad

    def antithetic_grad(self, params: np.ndarray):
        grad = np.zeros(params.shape)

        # generate noise with different methods
        #noise = self.independent_gau(self.N)
        noise = self.orth_gau()
        #noise = self.hadamard()
        #noise = self.random_givens()

        for i in range(self.N):
            curr_noise = noise[i]
            theta1 = params + self.sigma * curr_noise
            theta2 = params - self.sigma * curr_noise
            F1, F2 = self.fun(theta1), self.fun(theta2)
            grad += (F1 - F2) * curr_noise
        grad = grad / (2 * self.N * self.sigma)
        return grad

    def FD_grad(self, params: np.ndarray) -> np.ndarray:
        grad = np.zeros(params.shape)
        F = self.fun(params)

        # generate noise with different methods
        #noise = self.independent_gau(self.N)
        #noise = self.orth_gau()
        #noise = self.hadamard()
        noise = self.random_givens()

        for i in range(self.N):
            curr_noise = noise[i, :]
            theta = params + self.sigma * curr_noise
            F1 = self.fun(theta)
            grad += (F1 - F) * curr_noise
        grad = grad / (self.N * self.sigma)
        # print(grad.shape)
        return grad

    # monte carlo based gradient
    def independent_gau(self,sigma=0.1):
        res = np.empty((self.N, self.params_shape))
        for i in range(self.N):
            res[i] = np.random.multivariate_normal(mean=np.zeros(self.params_shape), cov=sigma * np.eye(self.params_shape))
        return res
    
    def orth_gau(self,sigma = 0.1):
        res = self.independent_gau()
        ort = orth(res.T)
        leng = np.linalg.norm(ort, axis=1, keepdims=True)
        return ort/leng

    
    def hadamard(self):
        T = self.T
        n = self.params_shape
        H1 = np.array([[1/np.sqrt(2),1/np.sqrt(2)],[1/np.sqrt(2),-1/np.sqrt(2)]])
        H = H1
        l = 2
        while 2*l<=n:
            H = np.kron(H,H1)
            l*=2
        n_h = H.shape[0]

        M = np.eye(n_h)
        for i in range(T):
            vec=np.random.choice([-1,1],n_h)
            D=np.diag(vec)
            temp = np.dot(H,D)
            M = np.dot(M,temp)
        #leng = np.power(n,-(T-1)/2)
        #leng = np.linalg.norm(M, axis=1, keepdims=True)
        return M


    def givens(self,n,i,j,theta):
        G = np.eye(n)
        G[i][i] = G[j][j] = np.cos(theta)
        G[i][j] = -np.sin(theta)
        G[j][i] = np.sin(theta)
        return G

    def random_givens(self):
        T = self.T
        n = self.params_shape
        K = np.eye(n)
        for t in range(T):
            i = np.random.randint(0,n)
            j = np.random.randint(0,n)
            theta = 2*np.pi*np.random.rand()
            K = np.dot(K,self.givens(n,i,j,theta))
        return K

    
    
    
    
    
    
    
    
    #optimization part
    def optimize(self):
        #initial = np.random.rand(self.params_shape)
        print(self.initial)
        opt = scipy.optimize.minimize(self.fun, self.initial, method='CG', jac=self.gradient_term,
                                      options={'disp': True})
        print(opt.x)
        return opt.x,-opt.fun,opt.nit,opt.nfev,opt.njev

In [ ]:
if __name__ == '__main__':
    df = pd.DataFrame(columns = ['fun','nit','nfev','njev'])
    for i in range(1000):
        # optimization part, conjugate gradient method
        sol = Solution(gym.make('CartPole-v1'),grad_method='vanilla')#change grad_method 
        params,fun_val,nit,nfev,njev = sol.optimize()
        df = df.append({'fun':fun_val,'nit':nit,'nfev':nfev,'njev':njev},ignore_index = True)
        params = params.reshape(sol.params_dot_shape)

        env = gym.make('CartPole-v1')
        obs = env.reset()
        a_list = []
        for t in range(max_num_step):
            #env.render()
            a = np.dot(params, obs)
           
            a = np.argmax(a.flatten())
           
            a_list.append(a)
          
            obs, reward, done, _ = env.step(a)  # take a random action
            if done:
                break
        #print('a_list',a_list)

(4,) 2
[0.40768703 0.05536604 0.78853488 0.28730518 0.45035059 0.30391231
 0.52639952 0.62381221]
         Current function value: -180.000000
         Iterations: 1
         Function evaluations: 26
         Gradient evaluations: 16
[0.66738736 0.28698635 0.67884208 0.28344279 0.29653929 0.48125098
 0.74241221 0.68486573]
